In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/humob/humob.csv")

In [4]:
# Rango de locaciones
x_min, x_max = 100, 150
y_min, y_max = 25, 125

# Marcar las filas que están dentro del rango
data['in_range'] = (data['x'] > x_min) & (data['x'] <= x_max) & (data['y'] > y_min) & (data['y'] <= y_max)

# Calcular si cada usuario/día estuvo exclusivamente dentro del rango
group_valid = data.groupby(['uid', 'd'])['in_range'].transform('all')

# Filtrar los datos
filtered_data = data[group_valid]
filtered_data = filtered_data.drop(columns=['in_range'])  # Eliminar la columna auxiliar

In [5]:
trajectory_length = 48

# Agregar una columna para `p_i`
filtered_data['p_i'] = 200 * (filtered_data['x'] - 1) + filtered_data['y']

# Crear listas para almacenar resultados
trajectories = []
user_indices = []
days = []

# Agrupar por `uid` y `d`
grouped = filtered_data.groupby(['uid', 'd'])

for (uid, day), group in grouped:
    if len(group) < 10:
        continue # Ignorar grupos con menos de 10 registros
    
    # Crear un array para toda la trayectoria inicializado en 'PAD'
    trajectory = np.full(trajectory_length, '[PAD]', dtype=object)
    
    # Asignar los valores de `p_i` según el índice de `t`
    trajectory[group['t'].values] = group['p_i'].astype(str).values
    
    # Guardar resultados
    trajectories.append(trajectory.tolist())
    user_indices.append(uid)
    days.append(day)

KeyboardInterrupt: 

In [7]:
df = pd.DataFrame({
    'trajectory': trajectories,
    'user_index': user_indices,
    'day': days
})

In [13]:
# Guardar como HDF5
h5_file = "humob.h5"
df.to_hdf(h5_file, key='data', mode='w')

C:\Users\alexs\AppData\Local\Temp\ipykernel_35476\1694227914.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['trajectory'], dtype='object')]

  df.to_hdf(h5_file, key='data', mode='w')


In [ ]:
df.count()